In [1]:
import tensorflow as tf

In [2]:
# 读取解析数据，mnist有他专用的API接口
from tensorflow.examples.tutorials.mnist import input_data

In [5]:
mnist = input_data.read_data_sets("E:\\AI\\data\\mnist\\input_data", one_hot = True)
# mnist数据集的目标值
# mnist.train.labels

# mnist每个图片的形状
mnist.train.images.shape

# # 通过批处理获取数据
# image, label = mnist.train.next_batch(10)

def full_connected_nn():
    """全连接层神经网络进行Mnist手写数字识别"""
    mnist = input_data.read_data_sets("E:\\AI\\data\\mnist\\input_data", one_hot = True)

    # 定义命名空间
    with tf.variable_scope("data"):
        # 定义特征值占位符
        x = tf.placeholder(tf.float32, [None, 784], name="feature")
        # 定义目标值占位符
        y_true = tf.placeholder(tf.int32, [None, 10], name="label")

    # 手写数字10个类别
    # 设计一层的神经网络，即最后一层，有十个目标值也就有10个神经元
    # 确定网络的参数weight[784, 10] bias[10]
    # 要进行全连接蹭的矩阵运算\[None, 784\] * \[784, 10\] + \[10\] = \[None, 10\]
    with tf.variable_scope("fc_model"):
        # 随机初始化权重以及偏置参数，要使用变量OP定义
        weight = tf.Variable(tf.random_normal([784, 10], mean=0.0, stddev=1.0), name="weight")

        bias = tf.Variable(tf.random_normal([10], mean=0.0, stddev=1.0), name="bias")

        # 全连接层的运算，10个神经元
        y_predict = tf.matmul(x, weight) + bias


    with tf.variable_scope("softmax_cross"):
        # 先进行网络输出的值的概率计算，再进行交叉熵的损失计算
        all_loss = tf.nn.softmax_cross_entropy_with_logits(labels=y_true, logits=y_predict, name="computer_loss")
    
         # 求出平均的交叉熵损失
        loss = tf.reduce_mean(all_loss)

    with tf.variable_scope("GD"):

        train_op = tf.train.GradientDescentOptimizer(0.1).minimize(loss)

    
    with tf.variable_scope("accuracy"):
        # 求出每个样本是否相等的一个列表
        equal_list = tf.equal(tf.argmax(y_true, 1), tf.argmax(y_predict, 1))

        # 计算相等样本的比例
        accuracy = tf.reduce_mean(tf.cast(equal_list, tf.float32))

    # 收集在tensorboard中观察的张量值
    # 数值型 --> scalar
    tf.summary.scalar("loss", loss)
    tf.summary.scalar("acc", accuracy)

    # 维度高的张量值
    tf.summary.histogram("w", weight)
    tf.summary.histogram("b", bias)

    # 合并变量
    merged = tf.summary.merge_all()

    # 创建保存模型的OP
    saver = tf.train.Saver()

    with tf.Session() as sess:
        # 初始化变量OP
        sess.run(tf.global_variables_initializer())

        # 创建events文件
        file_writer = tf.summary.FileWriter("C:\\Users\\Administrator\\Git\\CFturb\\Deep_Learning\\tensorboard",                                                           graph=sess.graph)
        
        # 加载本地模型继续训练或者拿来进行预测测试集
        # 加载模型，从模型当中找出与当前训练的模型代码当中（名字一样的OP操作），覆盖原来的值
        ckpt = tf.train.latest_checkpoint("C:\\Users\\Administrator\\Git\\CFturb\\Deep_Learning\\案例\\")

        # 判断模型是否存在
        if ckpt:
            saver.restore(sess, ckpt)

        # 循环训练
        for i in range(3000):
            # 每批次提供50个
            mnist_x, mnist_y = mnist.train.next_batch(50)
            _, loss_result, acc_run, summary = sess.run([train_op, loss, accuracy, merged], 
                                               feed_dict = {x: mnist_x, y_true: mnist_y})

            print("第%d步训练的损失为：%f，50个样本的准确率为：%f" % (i, loss_result, acc_run))

        # 写入运行结果到文件中
        file_writer.add_summary(summary, i)

        # 每隔100步保存一次模型的参数
        if i % 100 == 0:
            
            saver.save(sess, "C:\\Users\\Administrator\\Git\\CFturb\\Deep_Learning\\案例")

    return None

if __name__ == '__main__':
    full_connected_nn()

Extracting E:\AI\data\mnist\input_data\train-images-idx3-ubyte.gz
Extracting E:\AI\data\mnist\input_data\train-labels-idx1-ubyte.gz
Extracting E:\AI\data\mnist\input_data\t10k-images-idx3-ubyte.gz
Extracting E:\AI\data\mnist\input_data\t10k-labels-idx1-ubyte.gz
Extracting E:\AI\data\mnist\input_data\train-images-idx3-ubyte.gz
Extracting E:\AI\data\mnist\input_data\train-labels-idx1-ubyte.gz
Extracting E:\AI\data\mnist\input_data\t10k-images-idx3-ubyte.gz
Extracting E:\AI\data\mnist\input_data\t10k-labels-idx1-ubyte.gz


InvalidArgumentError: 2 root error(s) found.
  (0) Invalid argument: You must feed a value for placeholder tensor 'data/feature' with dtype float and shape [?,784]
	 [[node data/feature (defined at <ipython-input-3-a8511a030e17>:18) ]]
	 [[softmax_cross/Mean/_19]]
  (1) Invalid argument: You must feed a value for placeholder tensor 'data/feature' with dtype float and shape [?,784]
	 [[node data/feature (defined at <ipython-input-3-a8511a030e17>:18) ]]
0 successful operations.
0 derived errors ignored.

Original stack trace for 'data/feature':
  File "C:\ProgramData\Anaconda3\envs\tensorflow\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\ProgramData\Anaconda3\envs\tensorflow\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\ProgramData\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\ProgramData\Anaconda3\envs\tensorflow\lib\site-packages\trait

### 现在正式开始案例的设计
注意，我们只设计一层的神经网络

1. mnist获取数据，特征值 \[None, 784\] ，目标值\[None, 10\]

2. 根据识别的类别数，建立全连接层神经网络模型

    * 权重：\[784, 10\]
    
    * 偏置：\[10\]

    * \[None, 784\] * \[784, 10\] + \[10\] = \[None, 10\]

这就是每次10个神经元输出结果

3. softmax、交叉熵损失计算API
    * API：`tf.nn.softmax_cross_entropy_with_logits(labels=None, logits=None, name=None)`
    * `tf.reduce_mean(input_tensor)`

In [4]:
4. 梯度下降优化

5. 开启会话开始训练